# Elo Numbers
This notebook is used to interactively play with the Elo calculations for a "Battle Royale" style game.

# Code
All the code for calculating the different elo score changes.


###Globals
Global values for the elo calculations.

In [0]:
MAX_PLAYER_LVL = 100
MAX_GAME_SIZE = 100  # in number of players
MAX_GAME_TIME = 100  # in minutes

SCALING_BONUS_BASE = 0.75
SCALING_BONUS_MOD = 0.25

PLACE_FINISHED_BASE = 51
PLACE_FINISHED_MULT = 2

DOWNED_MULT = -10

KILLS_MULT = 5
KILLS_MOD = 5

ASSISTS_MULT = 5
ASSISTS_MOD = 5
ASSISTS_DIV = 2

DEATH_PENALTY = -75

ACCURACY_BASE = 0.95
ACCURACY_MULT = 0.1

VICTORY_BONUS = 100

STAYED_BONUS = 25

LOSS_BASE = 1
LOSS_MULT = 0.04

TIME_SURV_BASE = 0.5

###Imports

In [0]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

###Main function to generate Elo score changes

In [0]:
def get_elo_change_scores(average_lvl, player_lvl, place_finished, kills, downs, assists, accuracy, time_survived,
                          game_length):
    """Function to calculate ELO change of a player with given game stats.

    Args:
        player_lvl: Level of player who's elo change is being calculated.

        average_level: Average level of players in match.

        place_finished: Place the player finished

        kills: Number of kills the player had.

        downs: Number of times the player was "downed" and had to be revived.

        assists: Number of assists the player had.

        accuracy: Percentage of shots hit.

        time_survived: Time in minutes the player was a live in the game.

    Returns:
        Dict of floats of the elo adjustments that would be made depending on if the player: lost, won,
        died and continued to spectate while team won, died and left the game while team won.

    """
    elo_change = {'lost': (), 'won_stayed': (), 'won_left': (), 'won_lived': ()}

    place_s = calc_place_score(place_finished)
    down_s = calc_down_score(downs)
    kill_s = calc_kill_score(kills)
    assist_s = calc_assist_score(assists)
    accuracy_s = calc_accuracy_score(accuracy)
    loss_mult = calc_loss_mult(average_lvl, player_lvl)
    time_mult = calc_time_survived_mult(time_survived, game_length)

    scaling_bonus = calc_scale_bonus(average_lvl, player_lvl)

    elo_change['lost'] = calc_elo(place_s, down_s, kill_s, assist_s, accuracy_s, loss_mult, time_mult, scaling_bonus,
                                  False, True, False)

    elo_change['won_stayed'] = calc_elo(place_s, down_s, kill_s, assist_s, accuracy_s, loss_mult, time_mult,
                                        scaling_bonus, True, True, True)

    elo_change['won_left'] = calc_elo(place_s, down_s, kill_s, assist_s, accuracy_s, loss_mult, time_mult,
                                      scaling_bonus, True, True, False)

    elo_change['won_lived'] = calc_elo(place_s, down_s, kill_s, assist_s, accuracy_s, loss_mult, time_mult,
                                       scaling_bonus, True, False, True)

    return elo_change

###Calculate Elo Function

In [0]:
def calc_elo(place_s, down_s, kill_s, assist_s, accuracy_s, loss_mult, time_mult, scaling_bonus, won, died, stayed):
    """Calcualtes the elo gained or lossed for a player based on scores of a single game.

    Args:
        place_s: Place score for given game.
        down_s: Downed score for a given game.
        kill_s: Kill score for a given game.
        assist_s: Assist score for a given game.
        accuracy_s: Accuracy score for a given game.
        loss_mult: Loss multiplier for a given game.
        time_mult: Time multiplier for a given game.
        scaling_bonus: Scaling bonus achieved for a given game.
        won: bool, true if the player won the game.
        died: bool, true if player died in the game
        stayed: bool, true if the player stayed till the end of the game.

    Globals:
        DEATH_PENALTY: Global of the elo penalty for dying.
        STAYED_BONUS: Global of the elo bonus for staying till the end of the game.
    """
    death_penalty = DEATH_PENALTY if died else 0
    stayed_bonus = STAYED_BONUS if stayed else -STAYED_BONUS

    score = 0
    score_with_bonus = 0

    if won:
        score = (((kill_s + assist_s) * accuracy_s) * time_mult) + (
            death_penalty + down_s + stayed_bonus + VICTORY_BONUS + place_s)
        score_with_bonus = (((kill_s + assist_s) * accuracy_s) * time_mult) + (
            death_penalty + down_s + stayed_bonus + place_s) + (VICTORY_BONUS * scaling_bonus)
    else:
        score = (((kill_s + assist_s) * accuracy_s) * time_mult) + place_s + (loss_mult * (down_s + DEATH_PENALTY))
        score_with_bonus = score

    return (score, score_with_bonus)

##Value calculations
Sub-sections, containing the value calculations for given score attributes.

###Time Survived Multiplier Calculation

In [0]:
def calc_time_survived_mult(time_survived, game_length):
    """Calculates the time survived multiplier for a player for a given game.

    Args:
        time_survived: The time the player survived in minutes.
        game_length: The length of the game in minutes.

    Globals:
        TIME_SURV_BASE: Base score for time multiplier value.

    Returns:
        Float: Calcualted time survived multiplier value.
    """
    return TIME_SURV_BASE + (time_survived / game_length)


###Loss Multiplier Calculation

In [0]:
def calc_loss_mult(average_lvl, player_lvl):
    """Calculates the loss multiplier for a given player.

    Args:
        average_lvl: Average level of players in a given game.
        player_lvl: Player level.

    Globals:
        LOSS_BASE: Base value of loss multiplier.
        LOSS_MULT: Multiplier value used to calculate loss multiplier
    
    Returns:
        Float: Calculated loss multiplier for a given player and game.
    """
    return LOSS_BASE + (LOSS_MULT * (player_lvl / average_lvl))

###Accuracy Score

In [0]:
def calc_accuracy_score(accuracy_percentage):
    """Calculates the 'Accuracy Score' based off of a given % accuracy.

    Args:
        accuracy_percentage: % accuracy of a player for a given game.

    Globals:
        ACCURACY_BASE: Global for the base value of the 'Accuracy Score'.
        ACCURACY_MULT: Global for the multiplier used in the 'Accuracy Score'.
        
    Returns:
        Float: Calculated Accuracy score between 0 and 1
    """
    return ACCURACY_BASE + (accuracy_percentage * ACCURACY_MULT)

###Assist Score Calculation

In [0]:
def calc_assist_score(assists):
    """Calculates the 'Assist Score' based off a given number of asssits.

    Args:
        assists: Number of assists a player had.

    Globals:
        ASSISTS_MULT: Global assists multiplier.
        ASSISTS_MOD: Global modifier for the 'Assist Score'.

    Returns:
        Int: Calculated 'Assist Score'.
    """
    return (((assists * ASSISTS_MULT) + ASSISTS_MOD) * assists) / ASSISTS_DIV

###Place Score Calculation

In [0]:
def calc_place_score(place_finished):
    """Calculates the 'Place Score' based on a given place.

    Args:
        place_finished: The place a player finished in a match.

    Globals: 
        PLACE_FINISHED_BASE: Global for the base score of the 'Place Score'.

    Returns:
        Int: Calculated 'Place score'.
    """
    return (PLACE_FINISHED_BASE - place_finished) * 2

###Kill Score Calculation

In [0]:
def calc_kill_score(kills):
    """Calculates the 'Kill Score' based on a given number of kills

    Args:
        kills: Number of kills.

    Globals:
        KILLS_MULT: Global kills multiplier value.
        KILLS_MOD: Global kills modifier value.

    Returns:
        Int: Calculated 'Kill Score'.
    """
    return ((kills * KILLS_MULT) + KILLS_MOD) * kills

###"Downed" Score Calculation

In [0]:
def calc_down_score(downs):
    """Calculates the 'Downed Score' based on a given number of times the player was downed.

    Args:
        downs: Number of times the player was downed in a game.

    Globals:
        DOWNED_MULT: Global downed multiplier.
    
    Returns:
        Int: Calculated 'Downed Score'
    """
    return downs * DOWNED_MULT

###Calculate Scale Bonus

In [0]:
def calc_scale_bonus(average_lvl, player_lvl):
    """Calcualtes the scale bonus of a player for a given game.

    Args:
        average_lvl: Average level of all players in a given game.
        player_lvl: Level of player the scale bonus is being calculated for.

    Globals:
        SCALING_BONUS_BASE: Global for base bonus value.
        SCALING_BONUS_MOD: Global modifier for bonus value.
    """
    return SCALING_BONUS_BASE + (SCALING_BONUS_MOD * (average_lvl / player_lvl))

#Calculated Elo Scores
Use the sliders below to update the calculated ELO values for a given player with the defined stats.

In [82]:
average_lvl = 25
player_lvl = 50
place_finished = 88
kills = 1
downs = 7
assists = 3
accuracy_percent = .25
time_survived = 3
game_length = 25



scores = get_elo_change_scores(average_lvl, player_lvl, place_finished, kills, downs, assists, accuracy_percent, time_survived, game_length)

print(f"Game Lost Elo Changes:\n    {scores.get('lost')[0]:.2f}\n")
print(f"Game Won and player died but stayed Elo Changes:\n    without modifier: {scores.get('won_stayed')[0]:.2f}, with modifier: {scores.get('won_stayed')[1]:.2f}\n")
print(f"Game Won and player died but left the game Elo Changes:\n    without modifier: {scores.get('won_left')[0]:.2f}, with modifier: {scores.get('won_left')[1]:.2f}\n")
print(f"Game Won and player lived Elo Changes:\n    without modifier: {scores.get('won_lived')[0]:.2f}, with modifier: {scores.get('won_lived')[1]:.2f}\n")


Game Lost Elo Changes:
    -206.42

Game Won and player died but stayed Elo Changes:
    without modifier: -69.82, with modifier: -82.32

Game Won and player died but left the game Elo Changes:
    without modifier: -119.82, with modifier: -132.32

Game Won and player lived Elo Changes:
    without modifier: 5.18, with modifier: -7.32



## Interactive Inputs
These cells only work if you are running the notebook locally.


In [80]:
team_size = 0

elo_scores = interact(get_elo_change_scores, average_lvl=(1,MAX_PLAYER_LVL), player_lvl=(1,MAX_PLAYER_LVL), 
                place_finished=(1,int(MAX_GAME_SIZE)), kills=(0,100), downs=(0,10), assists=(0,MAX_GAME_SIZE), 
                accuracy=(0.0,1.0,0.01), time_survived=(0,MAX_GAME_TIME), game_length=(0, MAX_GAME_TIME))

display(elo_scores)

{'lost': (28559.5, 28559.5),
 'won_left': (28639.5, 28639.5),
 'won_lived': (28764.5, 28764.5),
 'won_stayed': (28689.5, 28689.5)}

<function __main__.get_elo_change_scores>